In [1]:
import kagglehub 
import pandas as pd

# Data loading function definition and testing

In [2]:
path_historical = f'{kagglehub.dataset_download("filipkin/steam-reviews")}/output.csv'
path_games = f'{kagglehub.dataset_download("filipkin/steam-reviews")}/output_steamspy.csv'  
path_latest = f'{kagglehub.dataset_download("tarasivaniv/steam-user-reviews-preprocessed-january-data")}/preprocessed_validationset.csv'

In [10]:
games = pd.read_csv(path_games)
games.drop(columns=['appid', 'owners'], inplace=True)
games.to_csv('games.csv', index=False, header=False)

In [3]:
def load_data(path_historical, path_latest, path_games):
    # ------------------------------------------------------------------------------------------------
    ### Function to load data from csv files. Files can either be downloaed externally 
    # from kagglehub or already downloaded and stored in the same directory as the script.
    # ------------------------------------------------------------------------------------------------

    reviews_historical = pd.read_csv(path_historical)
    reviews_latest = pd.read_csv(path_latest)
    games = pd.read_csv(path_games)
    reviews_all = pd.concat([reviews_historical, reviews_latest])
    reviews_all = reviews_all[['id', 'app_id', 'content', 'author_id', 'is_positive']]
    combined_data = pd.merge(reviews_all, games,
                   left_on='app_id',
                   right_on='appid',
                   how='inner')
    combined_data.dropna(inplace=True)
    combined_data.drop_duplicates(inplace=True)
    return combined_data

In [4]:
combined_data = load_data(path_historical, path_latest, path_games)
combined_data.head()

,id,app_id,content,author_id,is_positive,appid,name,owners
0,181398278,630,my friend fucking glitched out of the map. dud...,76561199715566328,Negative,630,Alien Swarm,"2,000,000 .. 5,000,000"
1,179764650,630,mid,76561199175172339,Negative,630,Alien Swarm,"2,000,000 .. 5,000,000"
2,177801117,630,"its just ok, valve games are not usually just ok",76561199197172151,Negative,630,Alien Swarm,"2,000,000 .. 5,000,000"
3,175959006,630,nobody is playing this game,76561198880505482,Negative,630,Alien Swarm,"2,000,000 .. 5,000,000"
4,175942905,630,this game isn't very fun it runs like ass and ...,76561199208923353,Negative,630,Alien Swarm,"2,000,000 .. 5,000,000"


# Dataset preprocessing function definition

In [5]:
import spacy
import contractions
import re

from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import numpy as np

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer


def preprocess_data(combined_data, rare_threshold=5, min_doc_freq=3, max_doc_freq_ratio=0.10, 
                   batch_size=None, n_process=None):
    """
    Preprocesses text data from Steam reviews through multiple cleaning and filtering steps.
    
    Parameters:
    -----------
    combined_data : pandas.DataFrame
        DataFrame containing the text data to process
    rare_threshold : int
        Minimum frequency for a token to not be considered rare
    min_doc_freq : int
        Minimum number of documents a term must appear in
    max_doc_freq_ratio : float
        Maximum percentage of documents a term can appear in
    batch_size : int or None
        Custom batch size for spaCy processing (calculated automatically if None)
    n_process : int or None
        Number of processes for parallel processing (calculated automatically if None)
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with original and processed text data
    """
    # Data validation and cleanup
    if combined_data.isnull().any().any():
        print("Missing values exist in the dataset.")
        combined_data = combined_data.dropna().copy()
        print(f"Missing values removed. Remaining rows: {len(combined_data)}")
    
    if combined_data.duplicated().any():
        print("Duplicates exist in the dataset.")
        combined_data = combined_data.drop_duplicates().copy()
        print(f"Duplicates removed. Remaining rows: {len(combined_data)}")
    else:
        print("No duplicates or missing values exist in the dataset.")
        combined_data = combined_data.copy()  # Create a copy to avoid modifying original

    # Basic text cleanup function
    def basic_cleanup(text):
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = contractions.fix(text)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return text

    print("Applying basic text cleanup...")
    combined_data['cleaned_text'] = [basic_cleanup(text) for text in tqdm(
        combined_data['content'],
        desc="Basic cleanup",
        unit="texts"
    )]
    
    # Configure spaCy processing
    print("Loading spaCy model...")
    nlp = spacy.load('en_core_web_sm', 
                    disable=['ner', 'textcat', 'entity_linker', 
                            'entity_ruler', 'sentencizer'])
    
    # Optimize batch size based on average text length
    if batch_size is None:
        avg_text_length = combined_data['cleaned_text'].str.len().mean()
        batch_size = max(100, int(1000000 / avg_text_length))
    
    if n_process is None:
        n_process = min(cpu_count() - 1, 4)  # Leave some cores for system
    
    print(f"Processing texts with batch size {batch_size} and {n_process} processes...")
    
    # Process in chunks with improved progress tracking
    chunk_size = 10000
    total_chunks = (len(combined_data) // chunk_size) + (1 if len(combined_data) % chunk_size > 0 else 0)
    all_tokens = []
    
    for i in range(0, len(combined_data), chunk_size):
        chunk_end = min(i + chunk_size, len(combined_data))
        print(f"Processing chunk {i//chunk_size + 1}/{total_chunks} ({chunk_end - i} texts)")
        
        texts = combined_data.iloc[i:chunk_end]['cleaned_text'].tolist()
        chunk_tokens = []
        
        # Process the chunk with proper progress tracking
        for doc in tqdm(nlp.pipe(texts, batch_size=batch_size, n_process=n_process), 
                        total=len(texts), desc=f"Chunk {i//chunk_size + 1}"):
            try:
                # Extract lemmas excluding stopwords, punctuation, and numbers
                tokens = [token.lemma_ for token in doc 
                         if not token.is_stop and not token.is_punct 
                         and not token.like_num and len(token.lemma_) > 1]
                chunk_tokens.append(tokens)
            except Exception as e:
                print(f"Error processing document: {e}")
                chunk_tokens.append([])
        
        all_tokens.extend(chunk_tokens)
        print(f"Completed chunk {i//chunk_size + 1}/{total_chunks}")
    
    # Add tokens to DataFrame
    combined_data['tokens'] = all_tokens
    
    # Handle empty token lists
    empty_tokens = combined_data['tokens'].apply(lambda x: len(x) == 0)
    if empty_tokens.any():
        print(f"Removing {empty_tokens.sum()} rows with failed processing...")
        combined_data = combined_data[~empty_tokens].copy()
    
    # Calculate token frequencies more efficiently
    print("Calculating token frequencies...")
    token_counter = Counter()
    for tokens in tqdm(combined_data['tokens'], desc="Counting tokens"):
        token_counter.update(tokens)
    
    # Identify rare words
    rare_words = {word for word, freq in token_counter.items() if freq < rare_threshold}
    
    # Calculate document frequencies directly from tokens
    print("Calculating document frequencies...")
    doc_freq = Counter()
    for tokens in tqdm(combined_data['tokens'], desc="Counting doc freqs"):
        # Count each token only once per document
        doc_freq.update(set(tokens))
    
    # Filter by document frequency
    num_docs = len(combined_data)
    min_docs = min_doc_freq
    max_docs = max_doc_freq_ratio * num_docs
    
    valid_words = {word for word, freq in doc_freq.items() 
                  if min_docs <= freq <= max_docs and word not in rare_words}
    
    # Apply final filtering
    print("Applying final token filtering...")
    combined_data['filtered_tokens'] = combined_data['tokens'].apply(
        lambda tokens: [t for t in tokens if t in valid_words]
    )
    
    # Create filtered text from filtered tokens
    combined_data['filtered_text'] = combined_data['filtered_tokens'].apply(
        lambda tokens: " ".join(tokens)
    )
    
    # Print statistics
    token_lengths = combined_data['filtered_tokens'].apply(len)
    print("\nProcessing Statistics:")
    print(f"Final dataset shape: {combined_data.shape}")
    print(f"Average tokens per text: {token_lengths.mean():.2f}")
    print(f"Min tokens: {token_lengths.min()}")
    print(f"Max tokens: {token_lengths.max()}")
    
    return combined_data

In [6]:
preprocessed_data = preprocess_data(combined_data)
preprocessed_data.head()

No duplicates or missing values exist in the dataset.
Applying basic text cleanup...


Basic cleanup: 100%|██████████| 191655/191655 [00:00<00:00, 208370.31texts/s]


Loading spaCy model...
Processing texts with batch size 7737 and 4 processes...
Processing chunk 1/20 (10000 texts)


Chunk 1: 100%|██████████| 10000/10000 [00:24<00:00, 405.76it/s]


Completed chunk 1/20
Processing chunk 2/20 (10000 texts)


...

Counting tokens: 100%|██████████| 184037/184037 [00:00<00:00, 999446.00it/s]


Calculating document frequencies...


Counting doc freqs: 100%|██████████| 184037/184037 [00:00<00:00, 836691.30it/s]


Applying final token filtering...

Processing Statistics:
Final dataset shape: (184037, 12)
Average tokens per text: 10.35
Min tokens: 0
Max tokens: 2000


,id,app_id,content,author_id,is_positive,appid,name,owners,cleaned_text,tokens,filtered_tokens,filtered_text
0,181398278,630,my friend fucking glitched out of the map. dud...,76561199715566328,Negative,630,Alien Swarm,"2,000,000 .. 5,000,000",my friend fucking glitched out of the map dude...,"[friend, fucking, glitche, map, dude]","[friend, fucking, glitche, map, dude]",friend fucking glitche map dude
1,179764650,630,mid,76561199175172339,Negative,630,Alien Swarm,"2,000,000 .. 5,000,000",mid,[mid],[mid],mid
2,177801117,630,"its just ok, valve games are not usually just ok",76561199197172151,Negative,630,Alien Swarm,"2,000,000 .. 5,000,000",its just ok valve games are not usually just ok,"[ok, valve, game, usually, ok]","[ok, valve, usually, ok]",ok valve usually ok
3,175959006,630,nobody is playing this game,76561198880505482,Negative,630,Alien Swarm,"2,000,000 .. 5,000,000",nobody is playing this game,"[play, game]",[],
4,175942905,630,this game isn't very fun it runs like ass and ...,76561199208923353,Negative,630,Alien Swarm,"2,000,000 .. 5,000,000",this game is not very fun it runs like ass and...,"[game, fun, run, like, ass, community, ready, ...","[fun, run, like, ass, community, ready, kick, ...",fun run like ass community ready kick soon mis...


In [7]:
import os
from typing import List, Dict, Any
from tqdm.auto import tqdm
import numpy as np

from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from qdrant_client import QdrantClient
from qdrant_client.http import models

In [10]:
def store_reviews_for_rag(
    dataframe: pd.DataFrame,
    collection_name: str = "steam_reviews",
    openai_api_key: str = None,
    embedding_model: str = "text-embedding-3-small",
    chunk_size: int = 1000,
    batch_size: int = 100
) -> Qdrant:
    """
    Process Steam reviews and store them in Qdrant for use in a langchain RAG pipeline.
    
    Args:
        dataframe: DataFrame containing Steam reviews
        collection_name: Name for the Qdrant collection
        openai_api_key: OpenAI API key (defaults to env variable)
        embedding_model: OpenAI embedding model to use
        chunk_size: Character length for text splitting (if needed)
        batch_size: Batch size for processing
        
    Returns:
        langchain.vectorstores.Qdrant: Configured Qdrant vector store for langchain
    """
    # Set up API key
    if openai_api_key is None:
        openai_api_key = os.environ.get("OPENAI_API_KEY")
        if not openai_api_key:
            raise ValueError("OpenAI API key required - provide as parameter or set OPENAI_API_KEY env variable")
    
    # Initialize langchain's OpenAI embeddings
    embeddings = OpenAIEmbeddings(
        model=embedding_model,
        openai_api_key=openai_api_key
    )
    
    # Initialize Qdrant client
    client = QdrantClient(host="localhost", port=6333)
    
    # Create or get collection
    vector_size = 1536  # Size for text-embedding-3-small
    try:
        client.get_collection(collection_name)
        print(f"Collection '{collection_name}' already exists")
    except Exception:
        client.create_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(
                size=vector_size,
                distance=models.Distance.COSINE
            )
        )
        print(f"Created new collection '{collection_name}'")
    
    # Create langchain documents from dataframe
    documents = []
    
    print(f"Processing {len(dataframe)} reviews...")
    for _, row in tqdm(dataframe.iterrows(), total=len(dataframe)):
        # Skip if filtered_text is missing
        if pd.isna(row['filtered_text']) or row['filtered_text'] == "":
            continue
            
        # Create a langchain Document with appropriate metadata
        doc = Document(
            page_content=row['filtered_text'],
            metadata={
                "game_name": row['name'],
                "review_id": str(row.name),  # Using index as review_id
            }
        )
        documents.append(doc)
    
    print(f"Created {len(documents)} langchain documents")
    
    # Optional: Split long reviews into smaller chunks for better retrieval
    # Only use this if your reviews are very long
    if chunk_size > 0:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=50,
            separators=["\n\n", "\n", ". ", " ", ""]
        )
        documents = text_splitter.split_documents(documents)
        print(f"Split into {len(documents)} chunks")
    
    # Create Qdrant vector store using langchain's integration
    print("Storing documents in Qdrant...")
    vector_store = Qdrant.from_documents(
        documents=documents,
        embedding=embeddings,
        collection_name=collection_name,
        url="http://localhost:6333",
        prefer_grpc=False,
        batch_size=batch_size,
    )
    
    print(f"Successfully stored {len(documents)} documents in Qdrant")
    return vector_store

In [11]:
rag_store = store_reviews_for_rag(preprocessed_data)

/var/folders/3m/gyqgx68n4v72zw8nb1r3jrj80000gn/T/ipykernel_24017/3734454712.py:30: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


Created new collection 'steam_reviews'
Processing 184037 reviews...


100%|██████████| 184037/184037 [00:02<00:00, 72838.08it/s]


Created 173367 langchain documents
Split into 172576 chunks
Storing documents in Qdrant...


KeyboardInterrupt: 